<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#A-tool-to-visualize-trajectory-frames:-ViewManager" data-toc-modified-id="A-tool-to-visualize-trajectory-frames:-ViewManager-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>A tool to visualize trajectory frames: ViewManager</a></span><ul class="toc-item"><li><span><a href="#Example---show-all-neighbours-of-a-structure" data-toc-modified-id="Example---show-all-neighbours-of-a-structure-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Example - show all neighbours of a structure</a></span></li><li><span><a href="#Example---show-PT-structures-with-5-largest-and-5-smallest-potential-energies" data-toc-modified-id="Example---show-PT-structures-with-5-largest-and-5-smallest-potential-energies-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Example - show PT structures with 5 largest and 5 smallest potential energies</a></span></li><li><span><a href="#Example---plot-an-eigenvector-of-SqRA" data-toc-modified-id="Example---plot-an-eigenvector-of-SqRA-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Example - plot an eigenvector of SqRA</a></span></li></ul></li><li><span><a href="#Classify-full-trajectories-in-cells-of-FullGrid" data-toc-modified-id="Classify-full-trajectories-in-cells-of-FullGrid-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Classify full trajectories in cells of FullGrid</a></span></li><li><span><a href="#Assigning-trajectories-to-a-FullGrid-cell" data-toc-modified-id="Assigning-trajectories-to-a-FullGrid-cell-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Assigning trajectories to a FullGrid cell</a></span><ul class="toc-item"><li><span><a href="#Test:-assigning-PT-to-a-FullGrid" data-toc-modified-id="Test:-assigning-PT-to-a-FullGrid-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Test: assigning PT to a FullGrid</a></span></li><li><span><a href="#Assigning-a-real-trajectory-fo-a-FullGrid" data-toc-modified-id="Assigning-a-real-trajectory-fo-a-FullGrid-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Assigning a real trajectory fo a FullGrid</a></span></li></ul></li></ul></div>

In [1]:
import sys
import os

try:
    os.chdir(r"/home/hanaz63/PAPER_MOLECULAR_ROTATIONS_2022/nobackup/molgri")
    sys.path.append(r"/home/hanaz63/PAPER_MOLECULAR_ROTATIONS_2022/nobackup")
except FileNotFoundError:
    os.chdir(r"D:\HANA\phD\PAPER_2022\molecularRotationalGrids")
    sys.path.append(r"D:\HANA\phD\PAPER_2022\molecularRotationalGrids")
    
import warnings
warnings.filterwarnings("ignore")

In [2]:
import nglview as nv
import MDAnalysis as mda
import numpy as np
import time
from scipy.sparse import csr_array
from numpy.typing import NDArray
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.display import display

from molgri.molecules.transitions import SimulationHistogram, MSM, SQRA
from molgri.plotting.molecule_plots import TrajectoryPlot
from molgri.molecules.parsers import FileParser, ParsedEnergy, XVGParser
from molgri.paths import PATH_OUTPUT_PT, PATH_OUTPUT_LOGGING, PATH_OUTPUT_ENERGIES
from molgri.space.fullgrid import FullGrid
from molgri.space.utils import k_argmin_in_array, k_argmax_in_array
from molgri.plotting.widgets import ViewManager, sync_all_views, display_all_views

## A tool to visualize trajectory frames: ViewManager

- in Jupyter, using nglview
- input: mda.Universe and one or more indices along the trajectory
- output: frames plotted sequentially or overlapping, color/opacity indicating a property, moving simutanously

In [3]:
fg_assigning = FullGrid("8", "12", "linspace(0.2, 1, 5)")
sh_pt = SimulationHistogram("H2O_H2O_0179", is_pt=True, second_molecule_selection="bynum 4:6",
                           full_grid=fg_assigning)

sh_traj = SimulationHistogram("H2O_H2O_0095_2000", is_pt=False, second_molecule_selection="bynum 4:6",
                             full_grid=fg_assigning)

### Example - show all neighbours of a structure

In [5]:
# EXAMPLE - neighbours

# display the structure assigned to cell 42
vm = ViewManager(sh_pt.trajectory_universe)
view1 = vm.plot_frames_overlapping(np.where(sh_pt.get_full_assignments==42))
display(view1)


# display its neighbours
vm.fresh_view()
view2 = vm.plot_frames_sequential(sh_pt.get_indices_neighbours_of_cell_i(42))


NGLWidget()

Box(children=(NGLWidget(layout=Layout(width='200px')), NGLWidget(layout=Layout(width='200px')), NGLWidget(layo…

### Example - show PT structures with 5 largest and 5 smallest potential energies

In [7]:
# EXAMPLE - 


# display
vm = ViewManager(sh_pt.trajectory_universe)
vm.fresh_view()

# magnitudes
magnitudes = sh_pt.get_magnitude_energy("Potential")


# 5 structures with largest energies
num_extremes = 10
argmax_index = k_argmax_in_array(magnitudes, num_extremes)
vm.set_color_magnitude(magnitudes)
view1 = vm.plot_frames_overlapping(argmax_index, opacities=0.8)

# 5 structures with smallest energies
vm.fresh_view()
argmin_index = k_argmin_in_array(magnitudes, num_extremes)
vm.set_color_magnitude(magnitudes)
view2 = vm.plot_frames_overlapping(argmin_index, opacities=0.8)

# show both
sync_all_views([view1, view2])
display_all_views([view1, view2])

Box(children=(NGLWidget(), NGLWidget()), layout=Layout(display='inline-flex', flex_flow='row wrap', width='100…

### Example - plot an eigenvector of SqRA

In [8]:
# EXAMPLE - eigenvector 0

# display
vm = ViewManager(sh_pt.trajectory_universe)
vm.fresh_view()

# magnitudes
#mch = MplColorHelper()
magnitudes = sh_pt.get_magnitude_ith_eigenvector(0)
vm.set_opacity_magnitude(magnitudes)



# largest absolute values
num_extremes = 20
argmax_index = k_argmax_in_array(np.abs(magnitudes), num_extremes)

vm.plot_frames_overlapping(argmax_index)
vm.view

NGLWidget()

## Classify full trajectories in cells of FullGrid

In [9]:
# find assignments to particular position class
selected_class = 4
in_this_position_class = sh_traj.get_indices_same_position(selected_class)
print(f"There are {len(in_this_position_class )} frames in this position class.")

vm = ViewManager(sh_traj.trajectory_universe)
vm.fresh_view()

vm.plot_frames_overlapping(sh_traj.get_indices_same_position(selected_class))
vm.view

There are 26 frames in this position class.


NGLWidget()

In [6]:
# find assignments to particular orientation class

#full_grid=FullGrid(b_grid_name="42", o_grid_name="15", t_grid_name="[0.2, 0.3, 0.4]")
my_sh = SimulationHistogram("H2O_H2O_0280", is_pt=True, second_molecule_selection="bynum 4:6",
                            use_saved = False)
            

#selected_class = 2
#in_this_position_class = my_sh.get_indices_same_orientation(selected_class)
#print(f"There are {len(in_this_position_class )} frames in this orientation class (only plotting a subset).")

vm = ViewManager(my_sh.trajectory_universe)
#vm.fresh_view()
#view1 = vm.plot_frames_overlapping(in_this_position_class, opacities=0.8)


# this would be per definition right orientations
vm.fresh_view()
third_orientation = np.arange(0, 25)
print(third_orientation)
view2 = vm.plot_frames_overlapping(third_orientation, opacities=0.8)

display(view2, gui=True)


Warning! Voronoi objects can only be created for >=4 points
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24]


NGLWidget()

In [11]:
# find assignments to one cell of full grid
selected_class = 250
in_this_class = sh_pt.get_indices_same_cell(selected_class)
print(f"There are {len(in_this_class )} frames in this cell.")

vm = ViewManager(sh_pt.trajectory_universe)
vm.fresh_view()
vm.plot_frames_overlapping(in_this_class, opacities=0.8)
vm.view

There are 14 frames in this cell.


NGLWidget()

In [ ]:

# TODO: enable expressing magnitude as opacity
# TODO: enable plotting only the most extrem values (most + and - for eigenvector)
# TODO: plot eigenvectors and see if they are sensible
# TODO: also plot 1D eigenvectors sorted by orientation/position
# TODO: network flow plot for strongest rates?

# TODO: make transition matrix determination faster and try it for a large matrix
# TODO: HF forcefield and calculations
# TODO: enable the same for plotting for simulations (real trajectories)

## Assigning trajectories to a FullGrid cell
- input: (pseudo)trajectory and a FullGrid of choice
- assign structures to position, orientation or full classes (cells of FullGrid) 


### Test: assigning PT to a FullGrid

- if assigning to its own fg, assignments should be 1:1
- if assigning to a smaller fg, an uniform number per cell is expected

In [ ]:
evaluation_fg = FullGrid("40", "42", "linspace(0.2, 1.5, 10)")

# this pt was generated with fg = FullGrid("40", "42", "linspace(0.2, 1.5, 10)")
my_pt_name = "H2O_H2O_0179"

sh = SimulationHistogram(my_pt_name, full_grid=evaluation_fg, second_molecule_selection="bynum 4:6")
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
assignments = sh.get_position_assignments()
natural_numbers = np.array(range(len(sh.full_grid.get_position_grid_as_array())))
natural_numbers = np.repeat(natural_numbers, sh.full_grid.b_rotations.get_N())

assert np.all(assignments == natural_numbers)

assignments = sh.get_quaternion_assignments()
natural_numbers = np.array(range(sh.full_grid.b_rotations.get_N()))
natural_numbers = np.tile(natural_numbers, len(sh.full_grid.get_position_grid_as_array()))

for el1, el2 in zip(assignments, natural_numbers):
    print(el1, el2)
    #pass

#assignments = sh.get_full_assignments()
#natural_numbers = np.array(range(len(assignments)))

#for el1, el2 in zip(assignments, natural_numbers):
#    print(el1, el2)

### Assigning a real trajectory fo a FullGrid

In [ ]:
my_fg = FullGrid("8", "12", "[0.3, 0.6, 0.9, 1.2]")
sh_traj = SimulationHistogram("H2O_H2O_0095_2000", full_grid=my_fg, second_molecule_selection="bynum 4:6")


In [ ]:
# most populated cells 
assignments = sh_traj.get_full_assignments()
my_indices, my_counts = np.unique(assignments.astype(int), return_counts=True)
most_populated_cells = my_indices[k_argmax_in_array(my_counts, 5)]
vm = ViewManager(sh.trajectory_universe)
for selected_class in most_populated_cells:
    vm.fresh_view()
    vm.plot_frames_overlapping(np.where(assignments==selected_class)[0][::10])
    display(vm.view)

In [ ]:
# are the most populated cells also the ones with lowest average energy?
# maybe not because the size of the cell strongly affects the population
all_popular_frames = []
full_assignments = sh_traj.get_full_assignments().astype(int)
my_indices, my_counts = np.unique(full_assignments, return_counts=True)
most_populated_cells = my_indices[k_argmax_in_array(my_counts, 3)]
for populated_cell in most_populated_cells:
    belongs_to_this = np.where(sh_traj.get_full_assignments()==populated_cell)[0]
    all_popular_frames.extend(belongs_to_this)

cell_volumes = np.array(sh_traj.full_grid.get_total_volumes())
volume_per_assignment = cell_volumes[full_assignments]

pot_energy = energies.get_energies("Potential")/volume_per_assignment
print("Popular frames energy", np.min(pot_energy[all_popular_frames]), np.average(pot_energy[all_popular_frames]),
     np.max(pot_energy[all_popular_frames]))
print("All frames energy", np.min(pot_energy), np.average(pot_energy),
     np.max(pot_energy))

In [ ]:
from molgri.plotting.transition_plots import TransitionPlot
my_msm = MSM(sh, energies)
my_tp = TransitionPlot(my_msm)
my_tp.plot_heatmap(save=False)

In [ ]:
my_tp.plot_its(save=False)